In [26]:
%matplotlib widget

import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.lines import Line2D
import yaml
from IPython.display import display  # to display variables in a "nice" way

try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

pd.options.display.max_rows = 10
pd.options.display.max_columns = 200

In [56]:
rs = 296312+302736

In [57]:
df_path = "responses_hw.csv" 
df = pd.read_csv(df_path)
display(df)

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,Punk,"Hiphop, Rap","Reggae, Ska","Swing, Jazz",Rock n roll,Alternative,Latino,"Techno, Trance",Opera,Movies,Horror,Thriller,Comedy,Romantic,Sci-fi,War,Fantasy/Fairy tales,Animated,Documentary,Western,Action,History,Psychology,Politics,Mathematics,Physics,Internet,PC,Economy Management,Biology,Chemistry,Reading,Geography,Foreign languages,Medicine,Law,Cars,Art exhibitions,Religion,"Countryside, outdoors",Dancing,Musical instruments,Writing,Passive sport,Active sport,Gardening,Celebrities,Shopping,Science and technology,Theatre,Fun with friends,Adrenaline sports,Pets,Flying,Storm,Darkness,Heights,Spiders,Snakes,Rats,Ageing,Dangerous dogs,Fear of public speaking,Smoking,Alcohol,Healthy eating,Daily events,Prioritising workload,Writing notes,Workaholism,Thinking ahead,Final judgement,Reliability,Keeping promises,Loss of interest,Friends versus money,Funniness,Fake,Criminal damage,Decision making,Elections,Self-criticism,Judgment calls,Hypochondria,Empathy,Eating to survive,Giving,Compassion to animals,Borrowed stuff,Loneliness,Cheating in school,Health,Changing the past,God,Dreams,Charity,Number of friends,Punctuality,Lying,Waiting,New environment,Mood swings,Appearence and gestures,Socializing,Achievements,Responding to a serious letter,Children,Assertiveness,Getting angry,Knowing the right people,Public speaking,Unpopularity,Life struggles,Happiness in life,Energy levels,Small - big dogs,Personality,Finding lost valuables,Getting up,Interests or hobbies,Parents' advice,Questionnaires or polls,Internet usage,Finances,Shopping centres,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating,Age,Height,Weight,Number of siblings,Gender,Hand,Education,Only child,Home Town Type,Home Type
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,5.0,4.0,2.0,5.0,4.0,4.0,1.0,5.0,5.0,3.0,1.0,2.0,1.0,5.0,1.0,3.0,3.0,5.0,3.0,5.0,3.0,3.0,3.0,3.0,5.0,3.0,1.0,1.0,1.0,1.0,5.0,3.0,3.0,2.0,1.0,5.0,5.0,1.0,4.0,4.0,2.0,5.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,5,3.0,1.0,3.0,2.0,never smoked,drink a lot,4.0,2.0,2.0,5.0,4.0,2.0,5.0,4.0,4.0,1.0,3.0,5.0,1.0,1.0,3.0,4.0,1.0,3.0,1.0,3.0,1,4.0,5.0,4.0,3.0,2.0,1.0,1.0,1.0,4,2.0,3,on time,never,3.0,4.0,3.0,4.0,3.0,4.0,3.0,5.0,1.0,1.0,3.0,5.0,5.0,1.0,4.0,5.0,1.0,4.0,3.0,2.0,3.0,4.0,3.0,few hours a day,3.0,4.0,5.0,3.0,3.0,1,3.0,20.0,163.0,48.0,1.0,female,right,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,4.0,1.0,3.0,1.0,4.0,4.0,2.0,1.0,1.0,5.0,2.0,2.0,4.0,3.0,4.0,1.0,3.0,5.0,4.0,1.0,4.0,1.0,3.0,4.0,5.0,2.0,4.0,4.0,5.0,1.0,1.0,4.0,4.0,5.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,2.0,4.0,2.0,5.0,1.0,1.0,1.0,2.0,1.0,1,1.0,3.0,1.0,4.0,never smoked,drink a lot,3.0,3.0,2.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,1.0,2.0,5.0,4.0,4.0,1.0,2.0,1,2.0,4.0,3.0,2.0,4.0,4.0,4.0,1.0,3,1.0,3,early,sometimes,3.0,4.0,4.0,4.0,4.0,2.0,4.0,2.0,2.0,5.0,4.0,4.0,4.0,1.0,4.0,3.0,5.0,3.0,4.0,5.0,3.0,2.0,3.0,few hours a day,3.0,4.0,1.0,4.0,2.0,5,2.0,19.0,163.0,58.0,2.0,female,right,college/bachelor degree,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,4.0,1.0,4.0,3.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,4.0,4.0,2.0,4.0,2.0,5.0,5.0,2.0,2.0,1.0,1.0,2.0,1.0,5.0,2.0,4.0,2.0,4.0,1.0,1.0,5.0,2.0,5.0,2.0,3.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0,1.0,1.0,4.0,2.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,2.0,tried smoking,drink a lot,3.0,1.0,2.0,5.0,3.0,5.0,3.0,4.0,5.0,1.0,5.0,2.0,4.0,1.0,3.0,5.0,4.0,4.0,1.0,5.0,5,5.0,4.0,2.0,5.0,3.0,2.0,5.0,5.0,1,3.0,3,late,sometimes,2.0,3.0,4.0,3.0,5.0,3.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,4.0,5.0,3.0,1.0,few hours a day,2.0,4.0,1.0,4.0,3.0,4,2.0,20.0,176.0,67.0,2.0,female,right,secondary school,no,city,block of flats
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,4.0,2.0,2.0,1.0,2.0,5.0,1.0,2.0,1.0,5.0,4.0,4.0,3.0,3.0,4.0,3.0,1.0,2.0,5.0,1.0,2.0,4.0,4.0,5.0,4.

In [58]:
data = {'Smoking': {'never smoked': 1, 'tried smoking':2, 'former smoker':3, 'current smoker':4},
        'Alcohol': {'never':1, 'social drinker': 2, 'drink a lot':3},
        'Punctuality': {'early':1, 'on time':2, 'late':3},
        'Lying': {'never':1, 'only to avoid hurting someone': 2, 'sometimes':3 ,'everytime it suits me':4},
        'Internet usage': {'no time at all':1, 'less than an hour a day':2, 'few hours a day':3, 'most of the day':4}      
       }
df = df.replace(data)

In [59]:
df = df.iloc[:,:-10]

In [60]:
X = df.values[:,:-10]
X

array([[5., 3., 2., ..., 3., 2., 3.],
       [4., 4., 2., ..., 4., 5., 3.],
       [5., 5., 2., ..., 3., 4., 5.],
       ...,
       [4., 3., 1., ..., 3., 5., 1.],
       [5., 3., 3., ..., 1., 5., 3.],
       [5., 5., 4., ..., 1., 2., 2.]])

In [61]:
col_ent = list(df.columns)[0:63]
col_pers = list(df.columns)[63:140]

In [70]:
np.random.seed(rs)
rem_ent = np.random.choice(col_ent,5)
rem_pers = np.random.choice(col_pers,10)
print(rem_ent)
print(rem_pers)

['Country' 'Geography' 'Latino' 'Slow songs or fast songs' 'Chemistry']
['Personality' 'Loneliness' 'Reliability' 'New environment' 'Fake'
 'Heights' 'Internet usage' 'Prioritising workload' 'Interests or hobbies'
 'Mood swings']


In [71]:
df_1 = df.iloc[:,0:63]
df_2 = df.iloc[:, 63:]

In [72]:
workdf_1 = df_1.drop(columns=rem_ent)
workdf_2 = df_2.drop(columns=rem_pers)

In [73]:
workdf_1

,Music,Dance,Folk,Classical music,Musical,Pop,Rock,Metal or Hardrock,Punk,"Hiphop, Rap","Reggae, Ska","Swing, Jazz",Rock n roll,Alternative,"Techno, Trance",Opera,Movies,Horror,Thriller,Comedy,Romantic,Sci-fi,War,Fantasy/Fairy tales,Animated,Documentary,Western,Action,History,Psychology,Politics,Mathematics,Physics,Internet,PC,Economy Management,Biology,Reading,Foreign languages,Medicine,Law,Cars,Art exhibitions,Religion,"Countryside, outdoors",Dancing,Musical instruments,Writing,Passive sport,Active sport,Gardening,Celebrities,Shopping,Science and technology,Theatre,Fun with friends,Adrenaline sports,Pets
0,5.0,2.0,1.0,2.0,1.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,5.0,4.0,2.0,5.0,4.0,4.0,1.0,5.0,5.0,3.0,1.0,2.0,1.0,5.0,1.0,3.0,3.0,5.0,3.0,5.0,3.0,3.0,5.0,3.0,1.0,1.0,1.0,1.0,5.0,3.0,3.0,2.0,1.0,5.0,5.0,1.0,4.0,4.0,2.0,5.0,4.0,4.0
1,4.0,2.0,1.0,1.0,2.0,3.0,5.0,4.0,4.0,1.0,3.0,1.0,4.0,4.0,1.0,1.0,5.0,2.0,2.0,4.0,3.0,4.0,1.0,3.0,5.0,4.0,1.0,4.0,1.0,3.0,4.0,5.0,2.0,4.0,4.0,5.0,1.0,4.0,5.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,2.0,4.0,2.0,5.0
2,5.0,2.0,2.0,4.0,5.0,3.0,5.0,3.0,4.0,1.0,4.0,3.0,5.0,5.0,1.0,3.0,5.0,3.0,4.0,4.0,2.0,4.0,2.0,5.0,5.0,2.0,2.0,1.0,1.0,2.0,1.0,5.0,2.0,4.0,2.0,4.0,1.0,5.0,5.0,2.0,3.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0,1.0,1.0,4.0,2.0,5.0,5.0,5.0,5.0
3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,4.0,2.0,2.0,1.0,2.0,5.0,2.0,1.0,5.0,4.0,4.0,3.0,3.0,4.0,3.0,1.0,2.0,5.0,1.0,2.0,4.0,4.0,5.0,4.0,1.0,3.0,1.0,2.0,3.0,5.0,4.0,2.0,5.0,1.0,5.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,4.0,3.0,1.0,2.0,1.0,1.0
4,5.0,4.0,3.0,4.0,3.0,5.0,3.0,1.0,2.0,5.0,3.0,2.0,1.0,2.0,2.0,2.0,5.0,4.0,4.0,5.0,2.0,3.0,3.0,4.0,4.0,3.0,1.0,4.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,5.0,3.0,3.0,2.0,3.0,1.0,4.0,4.0,1.0,3.0,1.0,3.0,1.0,4.0,3.0,3.0,3.0,2.0,4.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,5.0,5.0,2.0,5.0,4.0,4.0,4.0,3.0,2.0,4.0,1.0,5.0,4.0,4.0,3.0,2.0,5.0,2.0,3.0,4.0,5.0,4.0,2.0,5.0,5.0,2.0,1.0,4.0,4.0,4.0,2.0,1.0,2.0,4.0,3.0,1.0,2.0,5.0,5.0,2.0,1.0,1.0,5.0,1.0,2.0,2.0,3.0,4.0,4.0,4.0,2.0,4.0,5.0,3.0,5.0,5.0,4.0,4.0
1006,4.0,5.0,1.0,4.0,1.0,4.0,1.0,1.0,4.0,1.0,1.0,2.0,3.0,1.0,4.0,1.0,5.0,2.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,5.0,2.0,5.0,4.0,1.0,1.0,5.0,4.0,5.0,5.0,2.0,2.0,1.0,2.0,1.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,NaN,1.0,5.0,5.0,1.0,2.0,5.0,1.0,5.0,1.0,5.0
1007,4.0,1.0,1.0,2.0,2.0,3.0,4.0,1.0,2.0,3.0,3.0,2.0,2.0,5.0,1.0,1.0,4.0,5.0,5.0,3.0,5.0,2.0,1.0,4.0,4.0,3.0,1.0,3.0,2.0,3.0,3.0,3.0,1.0,3.0,4.0,1.0,1.0,4.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,5.0
1008,5.0,3.0,3.0,3.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,3.0,2.0,5.0,2.0,1.0,5.0,3.0,4.0,5.0,3.0,1.0,2.0,5.0,5.0,3.0,1.0,4.0,3.0,5.0,3.0,1.0,1.0,5.0,4.0,3.0,3.0,2.0,5.0,3.0,1.0,1.0,5.0,1.0,5.0,4.0,1.0,4.0,2.0,4.0,1.0,3.0,5.0,5.0,5.0,5.0,1.0,4.0


In [74]:
workdf_2

,Flying,Storm,Darkness,Spiders,Snakes,Rats,Ageing,Dangerous dogs,Fear of public speaking,Smoking,Alcohol,Healthy eating,Daily events,Writing notes,Workaholism,Thinking ahead,Final judgement,Keeping promises,Loss of interest,Friends versus money,Funniness,Criminal damage,Decision making,Elections,Self-criticism,Judgment calls,Hypochondria,Empathy,Eating to survive,Giving,Compassion to animals,Borrowed stuff,Cheating in school,Health,Changing the past,God,Dreams,Charity,Number of friends,Punctuality,Lying,Waiting,Appearence and gestures,Socializing,Achievements,Responding to a serious letter,Children,Assertiveness,Getting angry,Knowing the right people,Public speaking,Unpopularity,Life struggles,Happiness in life,Energy levels,Small - big dogs,Finding lost valuables,Getting up,Parents' advice,Questionnaires or polls,Finances,Shopping centres,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating
0,1.0,1.0,1.0,1.0,5,3.0,1.0,3.0,2.0,1.0,3.0,4.0,2.0,5.0,4.0,2.0,5.0,4.0,1.0,3.0,5.0,1.0,3.0,4.0,1.0,3.0,1.0,3.0,1,4.0,5.0,4.0,2.0,1.0,1.0,1.0,4,2.0,3,2.0,1.0,3.0,4.0,3.0,4.0,3.0,5.0,1.0,1.0,3.0,5.0,5.0,1.0,4.0,5.0,1.0,3.0,2.0,4.0,3.0,3.0,4.0,5.0,3.0,3.0,1,3.0
1,1.0,1.0,1.0,1.0,1,1.0,3.0,1.0,4.0,1.0,3.0,3.0,3.0,4.0,5.0,4.0,1.0,4.0,3.0,4.0,3.0,1.0,2.0,5.0,4.0,4.0,1.0,2.0,1,2.0,4.0,3.0,4.0,4.0,4.0,1.0,3,1.0,3,1.0,3.0,3.0,4.0,4.0,2.0,4.0,2.0,2.0,5.0,4.0,4.0,4.0,1.0,4.0,3.0,5.0,4.0,5.0,2.0,3.0,3.0,4.0,1.0,4.0,2.0,5,2.0
2,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,2.0,2.0,3.0,3.0,1.0,5.0,3.0,5.0,3.0,5.0,1.0,5.0,2.0,1.0,3.0,5.0,4.0,4.0,1.0,5.0,5,5.0,4.0,2.0,3.0,2.0,5.0,5.0,1,3.0,3,3.0,3.0,2.0,3.0,5.0,3.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,4.0,4.0,4.0,3.0,3.0,4.0,3.0,1.0,2.0,4.0,1.0,4.0,3.0,4,2.0
3,2.0,1.0,1.0,5.0,5,5.0,4.0,5.0,5.0,3.0,3.0,3.0,4.0,4.0,5.0,3.0,1.0,4.0,5.0,2.0,1.0,5.0,5.0,5.0,5.0,4.0,3.0,3.0,1,1.0,2.0,5.0,5.0,1.0,5.0,4.0,3,3.0,1,1.0,2.0,1.0,3.0,1.0,3.0,3.0,2.0,5.0,5.0,4.0,5.0,3.0,3.0,2.0,2.0,1.0,1.0,1.0,2.0,4.0,2.0,4.0,3.0,3.0,4.0,4,1.0
4,1.0,2.0,1.0,1.0,1,2.0,2.0,4.0,3.0,2.0,2.0,4.0,3.0,2.0,3.0,5.0,5.0,4.0,2.0,3.0,3.0,1.0,3.0,5.0,5.0,5.0,1.0,3.0,1,3.0,3.0,4.0,5.0,3.0,4.0,5.0,3,3.0,3,2.0,4.0,3.0,3.0,3.0,3.0,3.0,5.0,4.0,2.0,3.0,5.0,5.0,2.0,3.0,5.0,3.0,2.0,4.0,3.0,3.0,4.0,3.0,4.0,3.0,3.0,2,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,1.0,2.0,4.0,2.0,2,2.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,1.0,4.0,2.0,3.0,3.0,4.0,4.0,4.0,2.0,1.0,5.0,2.0,4.0,1.0,3.0,1,3.0,5.0,3.0,5.0,4.0,1.0,3.0,3,2.0,3,3.0,4.0,2.0,4.0,4.0,3.0,4.0,3.0,2.0,2.0,4.0,5.0,4.0,4.0,4.0,3.0,3.0,4.0,5.0,4.0,3.0,3.0,5.0,4.0,3.0,4.0,3,4.0
1006,1.0,1.0,1.0,1.0,1,1.0,2.0,5.0,5.0,1.0,2.0,3.0,1.0,1.0,5.0,5.0,5.0,4.0,1.0,2.0,1.0,3.0,5.0,5.0,5.0,5.0,1.0,2.0,1,4.0,3.0,4.0,1.0,3.0,4.0,5.0,3,3.0,3,1.0,3.0,5.0,3.0,3.0,3.0,5.0,5.0,4.0,1.0,2.0,5.0,1.0,1.0,4.0,4.0,3.0,3.0,1.0,4.0,3.0,3.0,1.0,1.0,1.0,1.0,5,3.0
1007,2.0,1.0,3.0,2.0,2,1.0,1.0,1.0,5.0,2.0,2.0,2.0,3.0,1.0,1.0,4.0,1.0,5.0,1.0,4.0,1.0,5.0,4.0,5.0,5.0,2.0,1.0,5.0,1,1.0,5.0,4.0,5.0,3.0,5.0,1.0,3,1.0,1,3.0,2.0,3.0,4.0,1.0,3.0,5.0,3.0,1.0,2.0,1.0,5.0,5.0,5.0,3.0,1.0,3.0,3.0,5.0,4.0,5.0,1.0,2.0,5.0,2.0,2.0,2,5.0
1008,2.0,3.0,2.0,3.0,3,3.0,3.0,5.0,5.0,2.0,2.0,3.0,3.0,5.0,1.0,3.0,4.0,4.0,5.0,3.0,3.0,1.0,5.0,5.0,3.0,5.0,1.0,4.0,3,3.0,4.0,5.0,5.0,4.0,3.0,2.0,3,3.0,3,3.0,3.0,3.0,5.0,3.0,2.0,5.0,3.0,5.0,4.0,4.0,5.0,3.0,5.0,3.0,2.0,2.0,1.0,5.0,3.0,3.0,3.0,2.0,1.0,5.0,3.0,3,3.0


In [78]:
workdf_tot = pd.concat((workdf_1, workdf_2), axis = 1)
workdf_tot

,Music,Dance,Folk,Classical music,Musical,Pop,Rock,Metal or Hardrock,Punk,"Hiphop, Rap","Reggae, Ska","Swing, Jazz",Rock n roll,Alternative,"Techno, Trance",Opera,Movies,Horror,Thriller,Comedy,Romantic,Sci-fi,War,Fantasy/Fairy tales,Animated,Documentary,Western,Action,History,Psychology,Politics,Mathematics,Physics,Internet,PC,Economy Management,Biology,Reading,Foreign languages,Medicine,Law,Cars,Art exhibitions,Religion,"Countryside, outdoors",Dancing,Musical instruments,Writing,Passive sport,Active sport,Gardening,Celebrities,Shopping,Science and technology,Theatre,Fun with friends,Adrenaline sports,Pets,Flying,Storm,Darkness,Spiders,Snakes,Rats,Ageing,Dangerous dogs,Fear of public speaking,Smoking,Alcohol,Healthy eating,Daily events,Writing notes,Workaholism,Thinking ahead,Final judgement,Keeping promises,Loss of interest,Friends versus money,Funniness,Criminal damage,Decision making,Elections,Self-criticism,Judgment calls,Hypochondria,Empathy,Eating to survive,Giving,Compassion to animals,Borrowed stuff,Cheating in school,Health,Changing the past,God,Dreams,Charity,Number of friends,Punctuality,Lying,Waiting,Appearence and gestures,Socializing,Achievements,Responding to a serious letter,Children,Assertiveness,Getting angry,Knowing the right people,Public speaking,Unpopularity,Life struggles,Happiness in life,Energy levels,Small - big dogs,Finding lost valuables,Getting up,Parents' advice,Questionnaires or polls,Finances,Shopping centres,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating
0,5.0,2.0,1.0,2.0,1.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,5.0,4.0,2.0,5.0,4.0,4.0,1.0,5.0,5.0,3.0,1.0,2.0,1.0,5.0,1.0,3.0,3.0,5.0,3.0,5.0,3.0,3.0,5.0,3.0,1.0,1.0,1.0,1.0,5.0,3.0,3.0,2.0,1.0,5.0,5.0,1.0,4.0,4.0,2.0,5.0,4.0,4.0,1.0,1.0,1.0,1.0,5,3.0,1.0,3.0,2.0,1.0,3.0,4.0,2.0,5.0,4.0,2.0,5.0,4.0,1.0,3.0,5.0,1.0,3.0,4.0,1.0,3.0,1.0,3.0,1,4.0,5.0,4.0,2.0,1.0,1.0,1.0,4,2.0,3,2.0,1.0,3.0,4.0,3.0,4.0,3.0,5.0,1.0,1.0,3.0,5.0,5.0,1.0,4.0,5.0,1.0,3.0,2.0,4.0,3.0,3.0,4.0,5.0,3.0,3.0,1,3.0
1,4.0,2.0,1.0,1.0,2.0,3.0,5.0,4.0,4.0,1.0,3.0,1.0,4.0,4.0,1.0,1.0,5.0,2.0,2.0,4.0,3.0,4.0,1.0,3.0,5.0,4.0,1.0,4.0,1.0,3.0,4.0,5.0,2.0,4.0,4.0,5.0,1.0,4.0,5.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,2.0,4.0,2.0,5.0,1.0,1.0,1.0,1.0,1,1.0,3.0,1.0,4.0,1.0,3.0,3.0,3.0,4.0,5.0,4.0,1.0,4.0,3.0,4.0,3.0,1.0,2.0,5.0,4.0,4.0,1.0,2.0,1,2.0,4.0,3.0,4.0,4.0,4.0,1.0,3,1.0,3,1.0,3.0,3.0,4.0,4.0,2.0,4.0,2.0,2.0,5.0,4.0,4.0,4.0,1.0,4.0,3.0,5.0,4.0,5.0,2.0,3.0,3.0,4.0,1.0,4.0,2.0,5,2.0
2,5.0,2.0,2.0,4.0,5.0,3.0,5.0,3.0,4.0,1.0,4.0,3.0,5.0,5.0,1.0,3.0,5.0,3.0,4.0,4.0,2.0,4.0,2.0,5.0,5.0,2.0,2.0,1.0,1.0,2.0,1.0,5.0,2.0,4.0,2.0,4.0,1.0,5.0,5.0,2.0,3.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0,1.0,1.0,4.0,2.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,2.0,2.0,3.0,3.0,1.0,5.0,3.0,5.0,3.0,5.0,1.0,5.0,2.0,1.0,3.0,5.0,4.0,4.0,1.0,5.0,5,5.0,4.0,2.0,3.0,2.0,5.0,5.0,1,3.0,3,3.0,3.0,2.0,3.0,5.0,3.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,4.0,4.0,4.0,3.0,3.0,4.0,3.0,1.0,2.0,4.0,1.0,4.0,3.0,4,2.0
3,5.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,4.0,2.0,2.0,1.0,2.0,5.0,2.0,1.0,5.0,4.0,4.0,3.0,3.0,4.0,3.0,1.0,2.0,5.0,1.0,2.0,4.0,4.0,5.0,4.0,1.0,3.0,1.0,2.0,3.0,5.0,4.0,2.0,5.0,1.0,5.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,4.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,5.0,5,5.0,4.0,5.0,5.0,3.0,3.0,3.0,4.0,4.0,5.0,3.0,1.0,4.0,5.0,2.0,1.0,5.0,5.0,5.0,5.0,4.0,3.0,3.0,1,1.0,2.0,5.0,5.0,1.0,5.0,4.0,3,3.0,1,1.0,2.0,1.0,3.0,1.0,3.0,3.0,2.0,5.0,5.0,4.0,5.0,3.0,3.0,2.0,2.0,1.0,1.0,1.0,2.0,4.0,2.0,4.0,3.0,3.0,4.0,4,1.0
4,5.0,4.0,3.0,4.0,3.0,5.0,3.0,1.0,2.0,5.0,3.0,2.0,1.0,2.0,2.0,2.0,5.0,4.0,4.0,5.0,2.0,3.0,3.0,4.0,4.0,3.0,1.0,4.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,5.0,3.0,3.0,2.0,3.0,1.0,4.0,4.0,1.0,3.0,1.0,3.0,1.0,4.0,3.0,3.0,3.0,2.0,4.0,2.0,1.0,1.0,2.0,1.0,1.0,1,2.0,2.0,4.0,3.0,2.0,2.0,4.0,3.0,2.0,3.0,5.0,5.0,4.0,2.0,3.0,3.0,1.0,3.0,5.0,5.0,5.0,1.0,3.0,1,3.0,3.0,4.0,5.0,3.0,4.0,5.0,3,3.0,3,2.0,4.0,3.0,3.0,3.0,3.0,3.0,5.0,4.0,2.0,3.0,5.0,5.0,2.0,3.0,5.0,3.0,2.0,4.0,3.0,